In [1]:
# In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world, 
# at varying distances from the equator. To accomplish this, you'll be utilizing a simple Python library, 
# the OpenWeatherMap API, and a little common sense to create a representative model of weather across world cities.
# The first requirement is to create a series of scatter plots to showcase the following relationships:

# Temperature (F) vs. Latitude
# Humidity (%) vs. Latitude
# Cloudiness (%) vs. Latitude
# Wind Speed (mph) vs. Latitude

# After each plot, add a sentence or two explaining what the code is analyzing.
# The second requirement is to run linear regression on each relationship. This time, separate the plots 
# into Northern Hemisphere (greater than or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 
# degrees latitude):

# Northern Hemisphere - Temperature (F) vs. Latitude
# Southern Hemisphere - Temperature (F) vs. Latitude
# Northern Hemisphere - Humidity (%) vs. Latitude
# Southern Hemisphere - Humidity (%) vs. Latitude
# Northern Hemisphere - Cloudiness (%) vs. Latitude
# Southern Hemisphere - Cloudiness (%) vs. Latitude
# Northern Hemisphere - Wind Speed (mph) vs. Latitude
# Southern Hemisphere - Wind Speed (mph) vs. Latitude

# After each pair of plots, take the time to explain what the linear regression is modeling. 
# For example, describe any relationships you notice and any other analysis you may have.
# Your final notebook must:

# Randomly select at least 500 unique (non-repeat) cities based on latitude and longitude.
# Perform a weather check on each of the cities using a series of successive API calls.
# Include a print log of each city as it's being processed with the city number and city name.
# Save a CSV of all retrieved data and a PNG image for each scatter plot.

In [2]:
# Import dependencies:

import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
from config import api_key
import time   #used for time.sleep

In [3]:
# Printed api key to ensure that it imported correctly
# print(api_key)

In [4]:
# Generate a random list of 500 unique cities with a reasonably even distribution across the globe

# Set a range for lat and lng that includes the whole Earth
# latitude range is -90 to 90, longitude range is -180 to 180
lats = []
longs = []

lats = np.random.uniform(-90, 90, 25)
longs = np.random.uniform(-180, 180, 25)

#print(lats)
#print(longs)

In [5]:
# zip lists together to form coordinates
lat_long = zip(lats, longs)
lat_long

In [6]:
#lat_long = pd.DataFrame(lat_long)
#lat_long

In [7]:
# Use citipy to find cities near coordinates

#Import citipy
from citipy import citipy

cities = []
countries = []

for lat_long in lat_long:
    city = citipy.nearest_city(lat_long[0], lat_long[1]).city_name
    country = citipy.nearest_city(lat_long[0], lat_long[1]).country_code
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Print the city count to confirm sufficient count
print(len(cities))
print(len(countries))

24
24


In [8]:
cities = [city.title() for city in cities]
countries = [country.upper() for country in countries]

In [9]:
# Build dataframe to hold data
weather_df = pd.DataFrame({"Cities" : cities, "Countries" : countries})
weather_df.head()

,Cities,Countries
0,Rikitea,PF
1,Castro,CL
2,Khatanga,RU
3,Plettenberg Bay,ZA
4,San Quintin,MX


In [10]:
# Check for NaNs
weather_df.isnull().sum()

Cities       0
Countries    0
dtype: int64

In [11]:
# Add places to hold data for each city

weather_df["Latitude"] = ""
weather_df["Longitude"] = ""
weather_df["Temperature (F)"] = ""
weather_df["Humidity"] = ""
weather_df["Cloudiness"] = ""
weather_df["Wind Speed (mph)"] = ""
weather_df.head()

,Cities,Countries,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,Rikitea,PF,,,,,,
1,Castro,CL,,,,,,
2,Khatanga,RU,,,,,,
3,Plettenberg Bay,ZA,,,,,,
4,San Quintin,MX,,,,,,


In [12]:
# Use OpenWeather to get weather data for generated list of cities, test on one city

base_url = "http://api.openweathermap.org/data/2.5/weather?"
city_name = "Minneapolis"
country_code = "US"
query_url = f"{base_url}q={city_name},{country_code}&appid={api_key}&units=imperial"
#print(query_url) - test url worked
minneapolis_weather = requests.get(query_url).json()
minneapolis_weather

{'coord': {'lon': -93.2638, 'lat': 44.98},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': -4.94,
  'feels_like': -15.68,
  'temp_min': -7.6,
  'temp_max': -4,
  'pressure': 1026,
  'humidity': 50},
 'visibility': 10000,
 'wind': {'speed': 6.91, 'deg': 330},
 'clouds': {'all': 40},
 'dt': 1612727547,
 'sys': {'type': 1,
  'id': 4984,
  'country': 'US',
  'sunrise': 1612704302,
  'sunset': 1612740567},
 'timezone': -21600,
 'id': 5037649,
 'name': 'Minneapolis',
 'cod': 200}

In [13]:
#test get lat
minneapolis_weather["coord"]["lat"]

44.98

In [14]:
#test get long
minneapolis_weather["coord"]["lon"]

-93.2638

In [15]:
#test get temp
minneapolis_weather["main"]["temp"]

-4.94

In [16]:
#test get humidity
minneapolis_weather["main"]["humidity"]

50

In [17]:
#test get cloudiness
minneapolis_weather["clouds"]["all"]
#cloudiness exists on a scale documented at https://openweathermap.org/weather-conditions

40

In [18]:
#test get windspeed
minneapolis_weather["wind"]["speed"]

6.91

In [19]:
my_params = {"appid" : api_key, "units" : "imperial"}

for index, row in weather_df.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    city = row["Cities"]
    country = row["Countries"]
    
    my_params['q'] = f'{city},{country}'
    
    #make request
    cities_weather = requests.get(base_url, params=my_params).json()
    try:
        weather_df.loc[index, "Latitude"] = cities_weather["coord"]["lat"]
    except KeyError:
        weather_df.loc[index, "Latitude"] = np.nan
        print("skip")
        continue
       
    weather_df.loc[index, "Longitude"] = cities_weather["coord"]["lon"]
    weather_df.loc[index, "Temperature (F)"] = cities_weather["main"]["temp"]
    weather_df.loc[index, "Humidity"] = cities_weather["main"]["humidity"]
    weather_df.loc[index, "Cloudiness"] = cities_weather["clouds"]["all"]
    weather_df.loc[index, "Wind Speed (mph)"] = cities_weather["wind"]["speed"]
        
    print(index, city)
    time.sleep(1)
        

0 Rikitea
1 Castro
2 Khatanga
3 Plettenberg Bay
4 San Quintin
5 Bengkulu
6 Bethel
7 Dingle
8 Avarua
skip
10 Tuktoyaktuk
11 Seydisehir
12 Moron
13 Saint-Philippe
14 Albany
15 Berlevag
16 Pevek
skip
18 Taltal
19 Thompson
20 Biskamzha
21 Harper
22 Hobart
23 Mahebourg


In [20]:
weather_df

,Cities,Countries,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,Rikitea,PF,-23.1203,-134.969,77.23,73,96,12.59
1,Castro,CL,-42.4721,-73.7732,79.81,34,0,8.05
2,Khatanga,RU,71.9667,102.5,-30.68,79,0,10.16
3,Plettenberg Bay,ZA,-34.0527,23.3716,64.72,77,83,5.53
4,San Quintin,MX,30.4833,-115.95,65.43,54,1,9.06
5,Bengkulu,ID,-3.8004,102.266,75.7,84,100,7.11
6,Bethel,US,41.3712,-73.414,28.18,86,90,4.61
7,Dingle,IE,52.1408,-10.2689,37.4,81,90,16.11
8,Avarua,CK,-21.2078,-159.775,84.2,79,20,13.8
9,Mataura,PF,NaN,,,,,


In [21]:
#Drop rows with missing values
weather_df.dropna(inplace=True)
weather_df.reset_index(drop=True, inplace=True)
weather_df

,Cities,Countries,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,Rikitea,PF,-23.1203,-134.969,77.23,73,96,12.59
1,Castro,CL,-42.4721,-73.7732,79.81,34,0,8.05
2,Khatanga,RU,71.9667,102.5,-30.68,79,0,10.16
3,Plettenberg Bay,ZA,-34.0527,23.3716,64.72,77,83,5.53
4,San Quintin,MX,30.4833,-115.95,65.43,54,1,9.06
5,Bengkulu,ID,-3.8004,102.266,75.7,84,100,7.11
6,Bethel,US,41.3712,-73.414,28.18,86,90,4.61
7,Dingle,IE,52.1408,-10.2689,37.4,81,90,16.11
8,Avarua,CK,-21.2078,-159.775,84.2,79,20,13.8
9,Tuktoyaktuk,CA,69.4541,-133.037,-25.6,75,75,13.8
